<a href="https://www.kaggle.com/code/pib73nl/notebookcc09c15adb?scriptVersionId=115874627" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<section 
         style=
         "background-image:
          url('https://storage.googleapis.com/kaggle-competitions/kaggle/44171/logos/header.png?t=2022-12-14-02-59-05'); 
          bacground-color:DarkCyan">
    <div align=center style="line-height:70px;color:white;font-size:24px;letter-spacing:5px">
        <b>SHIFT CV WINTER 2023</b>
    </div>
    <div align=center style="line-height:40px;color:white;font-size:20px;letter-spacing:5px">
        <b>Бинарная классификация размытых изображений</b>
    </div>
    <hr>
    <div align=right style="color:white;font-weight:400;font-size:16px;letter-spacing:2px">
        Выполнил: Попович И.Б.
    </div>
</section>

In [1]:
import numpy as np
import pandas as pd 
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets

pd.set_option('display.max_colwidth', 200)
print('Tensorflow version - ', tf.__version__)

2023-01-09 04:36:12.088896: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2023-01-09 04:36:12.089043: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Tensorflow version -  2.4.1


## Проблемная область

<p><font size='4'>Задача заключается в определении факта размытости изображения. Имеем следующие данные:
    <ul>
        <li>2664 тренировочных изображения, для которых известен ответ</li>
        <li> 774 изображения для тестирования модели</li>
    </ul>
    Для решения задачи используем <b>TensorFlow</b>. Данные организуем в конвейер при помощи <code>tf.data.Dataset</code>. Расчет будем запускать на тензорных процессорах (<b>TPU</b>).
    </font>
</p> 

In [2]:
# для расчета используем тензорный процессор
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
    
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
#     strategy = tf.distribute.MirroredStrategy()
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470


2023-01-09 04:36:19.246386: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-09 04:36:19.249208: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2023-01-09 04:36:19.249245: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-09 04:36:19.249273: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (268c8d5600e8): /proc/driver/nvidia/version does not exist
2023-01-09 04:36:19.252564: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

REPLICAS:  8


In [3]:
# для тензорного процессора необходимы пути Google cloud storage
GCS_DS_PATH = KaggleDatasets().get_gcs_path() # you can list the bucket with "!gsutil ls $GCS_DS_PATH"
GCS_TRAIN_PATH = GCS_DS_PATH + '/train/train/'
GCS_TEST_PATH = GCS_DS_PATH + '/test/test/'
WITH_VALIDATION = False # делим датасет на train и valid (True) или все данные в train (False)

## Подготовка списка файлов для обучения

In [4]:
files = pd.read_csv(GCS_DS_PATH + '/train.csv', dtype={'filename': str, 'blur':np.uint8})

In [5]:
# в датасете 2664 изображения, поделенные на два класса; классы сбалансированы почти поровну
print(files.info())
print(files.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2664 entries, 0 to 2663
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  2664 non-null   object
 1   blur      2664 non-null   uint8 
dtypes: object(1), uint8(1)
memory usage: 23.5+ KB
None
              blur
count  2664.000000
mean      0.486862
std       0.499921
min       0.000000
25%       0.000000
50%       0.000000
75%       1.000000
max       1.000000


In [6]:
# добавим gcs-пути к именам файлов
files['filename'] = files['filename'].apply(lambda x: GCS_TRAIN_PATH + str(x))

In [7]:
files.head()

,filename,blur
0,gs://kds-70c503abd5ae4b2998787cd4483fa2b113153fe0d5c7446f5deb9ffe/train/train/kagouracdzwrjjxzzedi.jpg,0
1,gs://kds-70c503abd5ae4b2998787cd4483fa2b113153fe0d5c7446f5deb9ffe/train/train/ahnamimqdfqoqdnozabc.jpg,0
2,gs://kds-70c503abd5ae4b2998787cd4483fa2b113153fe0d5c7446f5deb9ffe/train/train/gwhdadvghuzinmzhzssx.jpg,0
3,gs://kds-70c503abd5ae4b2998787cd4483fa2b113153fe0d5c7446f5deb9ffe/train/train/onqwabwwckubrydgbzly.jpg,0
4,gs://kds-70c503abd5ae4b2998787cd4483fa2b113153fe0d5c7446f5deb9ffe/train/train/ewpqdruddbokqyzzupcw.jpg,1


In [8]:
label = files.pop('blur') # метки в отдельный фатафрейм

In [9]:
# если просили валидацию - делим
if WITH_VALIDATION:
    X_train, X_val, y_train, y_val = train_test_split(files, label, test_size = 0.05, random_state=42, stratify=label)
else:
    X_train = files
    y_train = label

In [10]:
IMAGE_SIZE = [640, 640]
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

NUM_TRAINING_IMAGES = X_train.shape[0]*2 # в двойном размере, т.к. каждое изображение еще аугментируем
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

## Подготовка конвейеров данных

In [11]:
def decode_image(*image_data):
    """
    Декодирует изображение
    
    Параметры:
     image_data: tuple (2); первый элемент - путь к файлу, второй - метка 
                 (ground truth для train и valid, имя файла для test)
    Возврат:
     image: тензор [*IMAGE_SIZE, 3] пикселей изображения, отображенный на интервал [0,1]
     label: метка изображения (ground truth для train и valid, имя файла для test)
    """
    
    image_path, label = image_data
    image = tf.io.read_file(tf.squeeze(image_path))
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) 

    return image, label

def decode_image_augm(*image_data):
    """
    Вносит некоторые изменения в изображение (аугментация)
    
    Параметры:
     image_data: tuple (2); первый элемент - путь к файлу, второй - метка 
                 (ground truth для train и valid, имя файла для test)
    Возврат:
     image: тензор [*IMAGE_SIZE, 3] пикселей изображения, отображенный на интервал [0,1]
     label: метка изображения (ground truth для train и valid, имя файла для test)
    """
    # на вм с TPU стоит версия tensorflow 2.4.1, поэтому аугментируем как-то так...
    image, label = decode_image(*image_data)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_contrast(image, 0.7, 1.5)
    image = tf.image.random_brightness(image, 0.3)
    image = tf.image.rot90(image, np.random.randint(1,4))
    return image, label


def load_dataset(filenames, labels, augmentation=False):
    """
    Готовит датасет из путей к файлам и меток, и маппирует на него функции декодирования изображений
    
    Параметры:
     filenames:    список путей к файлам
     labels:       список меток (ground truth для train и valid, имя файла для test)
     augmentation: признак применения аугментации (default - False)
    Возврат:
     dataset:      датасет (image, label)
    """
    
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

    dataset = dataset.map(decode_image_augm if augmentation else decode_image,
                          num_parallel_calls=tf.data.AUTOTUNE)
    
    return dataset

def get_dataset(filenames, labels, type_ds: ('train', 'valid', 'test') = 'train', augmentation=False):
    """
    Подготовка конвейеров
    
    Параметры:
     filenames:    список путей к файлам
     labels:       список меток (ground truth для train и valid, имя файла для test)
     type_ds:      тип датасета (один из вариантов: 'train', 'valid', 'test')
     augmentation: признак применения аугментации (default - False)
    Возврат:
     dataset:      датасет (image, label)
    """
    assert type_ds in get_dataset.__annotations__['type_ds']
    
    dataset = load_dataset(filenames, labels, augmentation=augmentation)
    
    if type_ds == 'train':
        dataset = dataset.repeat() # the training dataset must repeat for several epochs
        dataset = dataset.shuffle(2048)
    
    dataset = dataset.batch(BATCH_SIZE)
    
    if type_ds == 'valid':
        dataset = dataset.cache()
    
    return dataset

In [12]:
# готовим конвейеры для train-а
training_dataset = get_dataset(X_train, y_train, 'train') # оригинальные изображения
augmented_dataset = get_dataset(X_train, y_train, 'train', augmentation=True) # аугментированные изображения
training_dataset = training_dataset.concatenate(augmented_dataset)

In [13]:
# ... и для валидации, если заказана
if WITH_VALIDATION:
    validation_dataset = get_dataset(X_val, y_val, 'valid') # оригинальные изображения
    augmented_dataset = get_dataset(X_val, y_val, 'valid', augmentation=True) # аугментированные изображения
    validation_dataset = validation_dataset.concatenate(augmented_dataset)

## Обучение модели

In [14]:
EPOCHS = 20

# организуем распределенное обучение на TPU
with strategy.scope():    
    pretrained_model = tf.keras.applications.DenseNet201(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    pretrained_model.trainable = False # обучение пока отключим
    
    # добавим немного слоев...
    model = tf.keras.Sequential([
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    # метрика в соревновании AUC_ROC
    auc_roc = tf.keras.metrics.AUC()
        
model.compile(
#     optimizer='adam',
    optimizer=tf.keras.optimizers.Nadam(),
    loss = 'binary_crossentropy',
    metrics=auc_roc
)

historical = model.fit(training_dataset, 
          steps_per_epoch=STEPS_PER_EPOCH, 
          epochs=EPOCHS, 
          validation_data=validation_dataset if WITH_VALIDATION else None)

74842112/74836368 [==============================] - 0s 0us/step
Epoch 1/20
41/41 [==============================] - 216s 3s/step - loss: 0.6832 - auc: 0.6582
Epoch 2/20
41/41 [==============================] - 121s 3s/step - loss: 0.3624 - auc: 0.9156
Epoch 3/20
41/41 [==============================] - 116s 3s/step - loss: 0.2108 - auc: 0.9696
Epoch 4/20
41/41 [==============================] - 118s 3s/step - loss: 0.2225 - auc: 0.9686
Epoch 5/20
41/41 [==============================] - 118s 3s/step - loss: 0.1578 - auc: 0.9807
Epoch 6/20
41/41 [==============================] - 126s 3s/step - loss: 0.1500 - auc: 0.9848
Epoch 7/20
41/41 [==============================] - 115s 3s/step - loss: 0.1416 - auc: 0.9860
Epoch 8/20
41/41 [==============================] - 117s 3s/step - loss: 0.1109 - auc: 0.9913
Epoch 9/20
41/41 [==============================] - 114s 3s/step - loss: 0.1247 - auc: 0.9884
Epoch 10/20
41/41 [==============================] - 117s 3s/step - loss: 0.1096 - auc: 0

In [15]:
with strategy.scope():    
    pretrained_model.trainable = True # немного "расшатаем" модель (эмпирическое наблюдение - иногда очень помогает :)

model.compile(
    optimizer=tf.keras.optimizers.Nadam(),
    loss = 'binary_crossentropy',
    metrics=auc_roc)
    
EPOCHS = 3

historical = model.fit(training_dataset, 
          steps_per_epoch=STEPS_PER_EPOCH, 
          epochs=EPOCHS, 
          validation_data=validation_dataset if WITH_VALIDATION else None)

Epoch 1/3
41/41 [==============================] - 406s 3s/step - loss: 0.8716 - auc: 0.9763
Epoch 2/3
41/41 [==============================] - 118s 3s/step - loss: 0.1059 - auc: 0.9927
Epoch 3/3
41/41 [==============================] - 120s 3s/step - loss: 0.1215 - auc: 0.9904


In [16]:
model.compile(
    optimizer=tf.keras.optimizers.Nadam(1e-5), # делаем fine-tuning с низким learning rate
    loss = 'binary_crossentropy',
    metrics=auc_roc)

EPOCHS = 10

historical = model.fit(training_dataset, 
          steps_per_epoch=STEPS_PER_EPOCH, 
          epochs=EPOCHS, 
          validation_data=validation_dataset if WITH_VALIDATION else None)

Epoch 1/10
41/41 [==============================] - 416s 3s/step - loss: 0.0416 - auc: 0.9947
Epoch 2/10
41/41 [==============================] - 122s 3s/step - loss: 0.0173 - auc: 0.9998
Epoch 3/10
41/41 [==============================] - 117s 3s/step - loss: 0.0138 - auc: 0.9995
Epoch 4/10
41/41 [==============================] - 118s 3s/step - loss: 0.0160 - auc: 0.9995
Epoch 5/10
41/41 [==============================] - 119s 3s/step - loss: 0.0086 - auc: 1.0000
Epoch 6/10
41/41 [==============================] - 120s 3s/step - loss: 0.0083 - auc: 1.0000
Epoch 7/10
41/41 [==============================] - 115s 3s/step - loss: 0.0086 - auc: 0.9993
Epoch 8/10
41/41 [==============================] - 120s 3s/step - loss: 0.0119 - auc: 0.9998
Epoch 9/10
41/41 [==============================] - 115s 3s/step - loss: 0.0063 - auc: 1.0000
Epoch 10/10
41/41 [==============================] - 115s 3s/step - loss: 0.0053 - auc: 1.0000


## Prediction and submission

In [17]:
# готовим конвейер для test-а
test_files_path = tf.io.gfile.glob(GCS_TEST_PATH + '*.jpg')
test_files_list = [os.path.split(path)[1] for path in test_files_path]
test_dataset = get_dataset(test_files_path, test_files_list, 'test')

2023-01-09 05:54:09.886980: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.


In [18]:
NUM_TEST_IMAGES = len(test_files_list)

In [19]:
%%time
# предсказание и подготовка submission.csv
print('Computing predictions...')
test_images_ds = test_dataset.map(lambda image, filename: image, num_parallel_calls=tf.data.AUTOTUNE)
predictions = model.predict(test_images_ds)

print('Generating submission.csv file...')
test_ids_ds = test_dataset.map(lambda image, filename: filename, num_parallel_calls=tf.data.AUTOTUNE).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
np.savetxt('submission.csv', np.rec.fromarrays([test_ids, np.squeeze(predictions)]), 
           fmt=['%s', '%.5f'], delimiter=',', header='filename,blur', comments='')

Computing predictions...
Cause: could not parse the source code of <function <lambda> at 0x7f99b2543560>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Generating submission.csv file...
Cause: could not parse the source code of <function <lambda> at 0x7f9990ed8830>: no matching AST found
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
CPU times: user 5.51 s, sys: 337 ms, total: 5.85 s
Wall time: 1min 6s
